In [70]:
import pandas as pd
import subprocess
from tqdm.auto import tqdm
import forgi

In [3]:
df_train = pd.read_json("../data/train.json",lines=True)

In [4]:
df_test = pd.read_json("../data/test.json", lines=True)

In [6]:
df = pd.concat([df_train,df_test])

In [11]:
for id_, seq in df[['id','sequence']].values:
    with open('./tmp/input.fasta','w') as f:
        f.write('>0\n'+seq)
    break

In [12]:
!cat ./tmp/input.fasta

>0
GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUAACUGGAAUAACCCAUACCAGCAGUUAGAGUUCGCUCUAACAAAAGAAACAACAACAACAAC

In [28]:
seq = "GGAAAACGUGGGUGCGCGAAGUCCUACGUACGUGAAACACGGGACGUACCGGCAAGCGGACGGACGUACUUUUGAUUCGUCAAAAGAAAAGAAACAACAACAACAAC"

In [42]:
my_envs = os.environ.copy()
my_envs['PATH'] = '/home/stepan/.local/bin:/home/stepan/perl5/bin:/home/stepan/perl5/bin:/media/stepan/files/projects/private/openVaccine/envs/graph/bin:/home/stepan/miniconda3/condabin:/home/stepan/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/home/stepan/.npm-global/bin:/home/stepan/.npm-global/bin'

In [52]:
cp = subprocess.run("RNAsubopt < ./tmp/input.fasta", shell=True,env=my_envs,capture_output=True)

In [65]:
cp.returncode

0

In [67]:
def sample_structures(sequence):
    with open('./tmp/input.fasta','w') as f:
        f.write('>0\n'+sequence)
    cp = subprocess.run("RNAsubopt < ./tmp/input.fasta", shell=True,env=my_envs,capture_output=True)
    assert cp.returncode == 0
    structures = [line.split( ) for line in str(cp.stdout).split('\\n')[2:-1]]
    return [{"structure": struct, "energy": energy} for struct, energy in structures]

In [96]:
def create_cg(sequence, structure):
    with open('./tmp/tmp_seq.dbn','w') as f:
        f.write('\n'.join([sequence, structure]))
    cgs = forgi.load_rna('./tmp/tmp_seq.dbn')
    assert len(cgs) == 1
    cg, = cgs
    return cg

In [98]:
cg = create_cg(sequence, structure)

In [73]:
def cg_to_d(cg):
    nodes = {}
    edges = []
    for name, items in cg.defines.items():
        segments = None
        if len(items) == 2:
            segments = [items]
        elif len(items) == 4:
            segments = [items[:2],items[2:]]
        elif len(items) == 0:
            segments = []
        else:
            print(name, items)
            assert False
        segments = [(seg[0]-1, seg[1]) for seg in segments]
        nodes[name] = segments
    for edge_start, edge_ends in cg.edges.items():
        for edge_end in edge_ends:
            edges.append((edge_start,edge_end))
    return {"nodes":nodes, "edges":edges}

In [74]:
sampled_structures = {}
for id_, sequence in tqdm(df[['id','sequence']].values):
    res = sample_structures(sequence)
    for d in res:
        structure = d['structure']
        cg = create_cg(sequence, structure)
        d.update(cg_to_d(cg))
    sampled_structures[id_] = res

In [76]:
import json

In [130]:
with open('../data/sampled_structures.json','w') as f:
    json.dump(sampled_structures, f)

In [121]:
import numpy as np

In [2]:
for recs in .values():
    loop_type = ['o']*len(recs['structure'])
    for node_name, spans in rec['nodes'].items():
        for span in spans:
            for i in range(*span):
                loop_type[i] = node_name[0]
    rec['loop_type'] = loop_type

NameError: name 'structures' is not defined

In [84]:
import seaborn as sns

In [94]:
from collections import Counter

In [95]:
Counter([len(v) for v in sampled_structures.values()],kde=False, log_scale=True).most_common()

[(1, 1235),
 (2, 882),
 (3, 584),
 (4, 559),
 (5, 511),
 (6, 328),
 (10, 219),
 (7, 181),
 (8, 171),
 (9, 161),
 (12, 121),
 (11, 112),
 (16, 103),
 (14, 85),
 (13, 81),
 (15, 67),
 (17, 55),
 (19, 52),
 (18, 49),
 (20, 47),
 (21, 38),
 (24, 31),
 (22, 30),
 (27, 24),
 (30, 24),
 (26, 19),
 (23, 18),
 (29, 14),
 (33, 13),
 (28, 12),
 (25, 12),
 (36, 12),
 (31, 12),
 (34, 10),
 (35, 10),
 (37, 10),
 (42, 8),
 (40, 8),
 (47, 7),
 (32, 5),
 (51, 5),
 (64, 4),
 (39, 4),
 (48, 4),
 (38, 4),
 (58, 4),
 (52, 4),
 (43, 4),
 (46, 4),
 (41, 4),
 (54, 4),
 (126, 3),
 (65, 3),
 (44, 3),
 (45, 3),
 (50, 3),
 (57, 3),
 (56, 3),
 (88, 2),
 (84, 2),
 (61, 2),
 (78, 2),
 (96, 2),
 (63, 2),
 (68, 2),
 (76, 2),
 (129, 1),
 (149, 1),
 (82, 1),
 (94, 1),
 (144, 1),
 (142, 1),
 (138, 1),
 (72, 1),
 (238, 1),
 (171, 1),
 (120, 1),
 (60, 1),
 (158, 1),
 (87, 1),
 (119, 1),
 (121, 1),
 (53, 1),
 (141, 1),
 (83, 1),
 (114, 1),
 (170, 1),
 (98, 1),
 (75, 1),
 (117, 1),
 (90, 1),
 (55, 1),
 (123, 1),
 (102, 1),
 